In [1]:
import uproot
import awkward as ak
from coffea import nanoevents
import numpy as np
import pandas as pd

In [2]:
filename = '/eos/uscms/store/user/lpcpfnano/emoreno/v2_2/2018/TTbar/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/TTToSemiLeptonic/211208_201500/0000/nano_mc2018_1-1.root'
f = uproot.open(filename)
events = nanoevents.NanoEventsFactory.from_root(f, "Events", entry_stop=1000, schemaclass=nanoevents.PFNanoAODSchema).events()

/uscms/home/aaportel/nobackup/mamba/envs/bh/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1912 branches) at 0x7fbd9a5b4400>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bh/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1912 branches) at 0x7fbd9a5b4400>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bh/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1912 branches) at 0x7fbd9a5b4400>, taking first instance
  warnings.warn(
/uscms/home/aaportel/nobackup/mamba/envs/bh/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1912 branc

In [3]:
from coffea.nanoevents.methods import candidate
def build_p4(cand):
    return ak.zip(
        {
            "pt": cand.pt,
            "eta": cand.eta,
            "phi": cand.phi,
            "mass": cand.mass,
            "charge": cand.charge,
        },
        with_name="PtEtaPhiMCandidate",
        behavior=candidate.behavior,
    )

In [4]:
# taus
loose_taus_mu = (events.Tau.pt > 20) & (abs(events.Tau.eta) < 2.3) & (events.Tau.idAntiMu >= 1)  # loose antiMu ID
loose_taus_ele = (
    (events.Tau.pt > 20)
    & (abs(events.Tau.eta) < 2.3)
    & (events.Tau.idAntiEleDeadECal >= 2)  # loose Anti-electron MVA discriminator V6 (2018) ?
)
n_loose_taus_mu = ak.sum(loose_taus_mu, axis=1)
n_loose_taus_ele = ak.sum(loose_taus_ele, axis=1)

muons = ak.with_field(events.Muon, 0, "flavor")
electrons = ak.with_field(events.Electron, 1, "flavor")

In [5]:
# muons
loose_muons = (
    (((muons.pt > 30) & (muons.pfRelIso04_all < 0.25)) | (muons.pt > 55))
    & (np.abs(muons.eta) < 2.4)
    & (muons.looseId)
)
n_loose_muons = ak.sum(loose_muons, axis=1)

good_muons = (
    (muons.pt > 30)
    & (np.abs(muons.eta) < 2.4)
    & (np.abs(muons.dz) < 0.1)
    & (np.abs(muons.dxy) < 0.05)
    & (muons.sip3d <= 4.0)
    & muons.mediumId
)
n_good_muons = ak.sum(good_muons, axis=1)

In [6]:
# electrons
loose_electrons = (
    (((electrons.pt > 38) & (electrons.pfRelIso03_all < 0.25)) | (electrons.pt > 120))
    & (np.abs(electrons.eta) < 2.4)
    & ((np.abs(electrons.eta) < 1.44) | (np.abs(electrons.eta) > 1.57))
    & (electrons.cutBased >= electrons.LOOSE)
)
n_loose_electrons = ak.sum(loose_electrons, axis=1)

good_electrons = (
    (electrons.pt > 38)
    & (np.abs(electrons.eta) < 2.4)
    & ((np.abs(electrons.eta) < 1.44) | (np.abs(electrons.eta) > 1.57))
    & (np.abs(electrons.dz) < 0.1)
    & (np.abs(electrons.dxy) < 0.05)
    & (electrons.sip3d <= 4.0)
    & (electrons.mvaFall17V2noIso_WP90)
)
n_good_electrons = ak.sum(good_electrons, axis=1)

In [7]:
# get candidate lepton
goodleptons = ak.concatenate([muons[good_muons], electrons[good_electrons]], axis=1)  # concat muons and electrons
# goodleptons = ak.concatenate(muons, electrons, axis=1)
goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]  # sort by pt

candidatelep = ak.firsts(goodleptons)  # pick highest pt

candidatelep_p4 = build_p4(candidatelep)  # build p4 for candidate lepton

In [8]:
# ak4 jets
ak4_jet_selector_no_btag = (
    (events.Jet.pt > 30) & (abs(events.Jet.eta) < 5.0) & events.Jet.isTight & (events.Jet.puId > 0)
)

goodjets = events.Jet[ak4_jet_selector_no_btag]

In [105]:
# fatjets
fatjets = events.FatJet

good_fatjets = (fatjets.pt > 200) & (abs(fatjets.eta) < 2.5) & fatjets.isTight & (fatjets.btagDeepB > .9)

good_fatjets = fatjets[good_fatjets]  # select good fatjets

two_fatjets = (ak.num(good_fatjets) >= 2) | (ak.num(good_fatjets) == 0)
candidate_fatjets = good_fatjets[two_fatjets]

candidate_fatjets = candidate_fatjets[ak.argsort(candidate_fatjets.pt, ascending=False)]  # sort them by pt

In [106]:
candidate_fatjets

<FatJetArray [[], [], [], [], ... [], [], [], []] type='983 * var * fatJet'>